In [1]:
import scrapy
import scrapy.crawler as crawler
from scrapy.utils.log import configure_logging
from multiprocessing import Process, Queue
from twisted.internet import reactor
from pydispatch import dispatcher
import logging
import json

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger('nlp')
logger.setLevel(logging.INFO)

# Fetch sentences

## USA goverment RSS channels

In [2]:
def fetch_rss_links():
    def f(q):
        try:
            links_storage = {'links' : []}
            class LinkCheckerSpider2(scrapy.Spider):
                name = "link_checker2"
                start_urls = ['https://www.state.gov/rss-feeds/']
                def parse(self, response):
                    a_selectors = response.xpath("//a")
                    for selector in a_selectors:
                        link = selector.xpath("@href").extract_first()
                        if 'rss' in link:
                            links_storage['links'].append(link)
            runner = crawler.CrawlerRunner()
            deferred = runner.crawl(LinkCheckerSpider2)
            deferred.addBoth(lambda _: reactor.stop())
            reactor.run()
            q.put(json.dumps(links_storage))
        except Exception as e:
            q.put(e)

    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    result = q.get()
    p.join()
    try:
        links_storage = json.loads(result)
    except:
        raise result
    return links_storage
#     print(links_storage)

# def get_data_from_links(links_storage):
    

usa_links_storage = fetch_rss_links()

{'links': ['https://www.state.gov/rss-feed/africa/feed/',
  'https://www.state.gov/rss-feed/arms-control-and-international-security/feed/',
  'https://www.state.gov/rss-feed/climate-environment-and-conservation/feed/',
  'https://www.state.gov/rss-feed/collected-department-releases/feed/',
  'https://www.state.gov/rss-feed/counterterrorism/feed/',
  'https://www.state.gov/rss-feed/democracy-human-rights-and-labor/feed/',
  'https://www.state.gov/rss-feed/department-press-briefings/feed/',
  'https://www.state.gov/rss-feed/diplomatic-security/feed/',
  'https://www.state.gov/rss-feed/direct-line-to-american-business/feed/',
  'https://www.state.gov/rss-feed/east-asia-and-the-pacific/feed/',
  'https://www.state.gov/rss-feed/economic-energy-agricultural-and-trade-issues/feed/',
  'https://www.state.gov/rss-feed/europe-and-eurasia/feed/',
  'https://www.state.gov/rss-feed/international-expositions/feed/',
  'https://www.state.gov/rss-feed/international-health-issues/feed/',
  'https://www

In [3]:
# class QuotesSpider(scrapy.Spider):
#     name = "quotes"
#     start_urls = ['https://www.state.gov/rss-feeds/']

#     def parse(self, response):
#         for quote in response.css('div.quote'):
#             print(quote.css('span.text::text').extract_first())

In [4]:
# runner = crawler.CrawlerRunner()
# deferred = runner.crawl(LinkCheckerSpider2)
# deferred.addBoth(lambda _: reactor.stop())
# reactor.run()
# q.put(None)

In [5]:
# intr = runner
# print(intr.settings)
# dir(intr)